# Run using AWS_BedRock_Anthropic_Claude_V2 Notebook template (Python 3.9 with numpy == 1.24.0 and refractml==1.0.3, Medium Container)


In [ ]:
!pip install openai==0.28.1

In [ ]:
AZURE_OPENAI_KEY=openai_key
AZURE_OPENAI_ENDPOINT="https://gear.openai.azure.com/"

In [ ]:
import os
import openai

openai.api_key = AZURE_OPENAI_KEY
openai.api_base = AZURE_OPENAI_ENDPOINT # your endpoint should look like the following https://YOUR_RESOURCE_NAME.openai.azure.com/
openai.api_type = 'azure'
openai.api_version = '2023-07-01-preview' # this might change in the future
deployment_name='gpt-35-turbo' #This will correspond to the custom name you chose for your deployment when you deployed a model. 

In [ ]:
message_text = [{"role":"system","content":"You are an AI assistant that helps people find information."},{"role":"user","content":"who is President of India ?"},{"role":"assistant","content":"As of September 2021, the Prime Minister of India is Narendra Modi."}]

In [ ]:
completion = openai.ChatCompletion.create(
  engine="gpt-35-turbo",
  messages = message_text,
  temperature=0.7,
  max_tokens=800,
  top_p=0.95,
  frequency_penalty=0,
  presence_penalty=0,
  stop=None
)

In [ ]:
completion

In [ ]:
completion['choices'][0]['message']['content'].replace('\n', '').replace(' .', '.').strip()

In [ ]:
from refractml import *
from refractml.constants import MLModelFlavours

# new score functions
from mosaic_utils.ai.score.base import ScoreBase
from typing import Tuple, Union, List, Any
import numpy as np

In [ ]:
# Model Registration
class ScoreTemplateExample(ScoreBase):
    """
    This Class Demonstrate How To Implements ScoreBase Interface Class And It Basic Usage.
    """    
    def __init__(self):
        super().__init__()
        import openai
        
        openai.api_key = self.AZURE_OPENAI_KEY
        openai.api_base = "https://gear.openai.azure.com/" #self.AZURE_OPENAI_ENDPOINT # your endpoint should look like the following https://YOUR_RESOURCE_NAME.openai.azure.com/
        openai.api_type = 'azure'
        openai.api_version = '2023-07-01-preview' # this might change in the future
        deployment_name='gpt-35-turbo'
    
    def request_processing_fn(self, request) :
        """
        Processes Request Object -> List[Input data, ..]. It could be:
               A List Mapping of All Value Can Be one of : 
                   - List[ [Feature_Value1, Feature_Value2, ...], [...] ]
                   - List[numpy.array(), numpy.array(), ...]
                   - List[tf.Tensor, tf.Tensor, tf.Tensor, ...]
                   - List[ SingleSample, SingleSample]
                   
        :return: (n_inputs, payload's)
        
        Warnings:
        1. Do not reshape your final output for single sample here, do it in prediction.
           Else payloads will be invalidated for extraction at raw and extraction level.
        """
        final_payload = []
        raw_payload = request.json["payload"]
        return (1, raw_payload) 
    
    def pre_processing_fn(self,payload):
        # All preprocessing step must occur in this section
        # Takes Single Sample -> Returns Single Sample
        
        # Not Doing Any Preprocessing Hence Returned payload
        #print("payload is ", payload)
        return payload

    def prediction_fn(self,
                      model: Any,
                      pre_processed_input 
                      ):
        """
                Does the main prediction on pre_processed_input(Single Sample) using supplied model .

                :param model: Supported Model
                :param pre_processed_input: Single Preprocessed Payload
                :return: Prediction Value From the model
                
                Important Notes:
                - Reshape your data array.reshape(1, -1) before predictions as it contains a single sample.
                    
        """
        #self.message_text 
        conversation=[{"role": "system", "content": "You are a helpful assistant."}]
        conversation.append({"role": "user", "content": pre_processed_input})
        #print (conversation)
        completion = openai.ChatCompletion.create(
            engine="gpt-35-turbo",
            messages = conversation,
            temperature=0.7,
            max_tokens=800,
            top_p=0.95,
            frequency_penalty=0,
            presence_penalty=0,
            stop=None
        )
        
        #print (completion)
        return completion['choices'][0]['message']['content'].replace('\n', '').replace(' .', '.').strip()


    class Meta:    
        # List of Callables() can be attached For Calling After AnSd Before Scoring
        def __init__(self):
            self.name = "Pre Hooked Me !"
            self.pre_call_hooks.append(self.print_)
        def print_(self):
            print(self.name)
        pre_call_hooks = []
        post_call_hooks = []

In [ ]:
prompt = "Who is the Defence Minister of India"
import requests
req = requests.Request()
req.json = {"payload":prompt}
req.json

In [ ]:
score_ = ScoreTemplateExample()
model_predictions = score_.score(None, req, dry_run=True)

In [ ]:
print(model_predictions[0].score_response)

In [ ]:
register_model(None,
               ScoreTemplateExample,
               "Azure_OpenAI_GPT35_Turbo",
               "Azure_OpenAI_GPT35_Turbo",
               MLModelFlavours.pytorch,
               init_script="pip install openai==0.28.1",
              )